In [15]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from skimage.feature import hog
from skimage.color import rgb2gray

In [19]:
# ========== Feature Extractor (Color Histogram) ==========
def extract_features(image, bins=32, resize_dim=(64, 64)):
    image = image.resize(resize_dim).convert('RGB') # Ensures that all images are RGB
    image_np = np.array(image)
    hist_r = np.histogram(image_np[:, :, 0], bins=bins, range=(0, 256))[0]
    hist_g = np.histogram(image_np[:, :, 1], bins=bins, range=(0, 256))[0]
    hist_b = np.histogram(image_np[:, :, 2], bins=bins, range=(0, 256))[0]
    hist = np.concatenate([hist_r, hist_g, hist_b])
    return hist / np.sum(hist)

In [20]:
def extract_hog_features(image, resize_dim=(64, 64), orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    image = image.resize(resize_dim).convert('RGB')
    gray = rgb2gray(np.array(image))
    features = hog(gray, orientations=orientations,
                   pixels_per_cell=pixels_per_cell,
                   cells_per_block=cells_per_block,
                   block_norm='L2-Hys')
    return features



def extract_combined_features(image):
    hog_feat = extract_hog_features(image)
    hist_feat = extract_features(image)
    return np.concatenate([hog_feat, hist_feat])

In [21]:
# Parameters
image_size = (64, 64)  # Resize all images to 64x64
valid_exts = ('.jpg', '.jpeg', '.png')

# Function to load images from a given folder
def load_dataset(root_dir, extractor_fn):
    X = []
    y = []
    class_names = sorted(os.listdir(root_dir))
    for label in class_names:
        label_path = os.path.join(root_dir, label)
        if not os.path.isdir(label_path):
            continue
        for fname in os.listdir(label_path):
            if fname.lower().endswith(valid_exts):
                try:
                    img_path = os.path.join(label_path, fname)
                    img = Image.open(img_path).convert('RGB')
                    if extractor_fn == None:
                        img = img.resize(image_size)
                        img_array = np.array(img).flatten()  # Flatten to 1D vector (64*64*3)
                        X.append(img_array)
                        y.append(label)
                    else:
                        features = extractor_fn(img)
                        X.append(features)
                        y.append(label)
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")
    return np.array(X), np.array(y)

# Load training and testing data
print("=== Load data ===")
func_name = extract_combined_features #extract_features  #extract_combined_features #None
X_train, y_train = load_dataset("Training", func_name)
X_test, y_test = load_dataset("Test", func_name)

# Encode labels (e.g., 'library' -> 0, etc.)
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

# Shuffle and standardize
X_train, y_train_enc = shuffle(X_train, y_train_enc, random_state=42)
X_test, y_test_enc = shuffle(X_test, y_test_enc, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
print(f"Classes: {le.classes_}")

=== Load data ===
Train shape: (15000, 1860), Test shape: (300, 1860)
Classes: ['library-indoor' 'museum-indoor' 'shopping_mall-indoor']


In [ ]:
# ---- SVM ---- Ignore for now
print("\n--- SVM Training ---")
svm_params = {'C': [1, 10], 'kernel': ['linear', 'rbf']}
svm = GridSearchCV(SVC(probability=True), svm_params, cv=3, scoring='accuracy')
svm.fit(X_train, y_train_enc)
svm_preds = svm.predict(X_test)
print(classification_report(y_test_enc, svm_preds))
print(confusion_matrix(y_test_enc, svm_preds))


--- SVM Training ---


In [ ]:
# ---- SVM ---- individual models
# === I need you to run this for different hyperparameters,
## I made an excel sheet where you can keep track of the results
## For now I think it is best to run SVM with func_name= extrac combined features (it is already there)
## https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict_proba
## The link above has the model details where u can see what are your options
### For now I believe C, kernel, and degree are the ones we can change, 
### if you think i'm missing an important hyperparameter, feel free to change it
print("\n--- SVM Training ---")
## The commented code below runs all combinations of models to come up with the best one but it will take forever
## Hence, just use a single model and change the values manually
# svm_params = {'C': [1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'degree': =[3,5,8]}
# svm_classifier = GridSearchCV(SVC(probability=True), svm_params, cv=3, scoring='accuracy')
svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42, degree= 8, probability=True)
svm_classifier.fit(X_train, y_train_enc)
print('=== Testing Performance ===')
svm_preds = svm_classifier.predict(X_test)
print(classification_report(y_test_enc, svm_preds))
print(confusion_matrix(y_test_enc, svm_preds))



--- SVM Training ---
              precision    recall  f1-score   support

           0       0.68      0.66      0.67       100
           1       0.75      0.72      0.73       100
           2       0.67      0.72      0.70       100

    accuracy                           0.70       300
   macro avg       0.70      0.70      0.70       300
weighted avg       0.70      0.70      0.70       300

[[66 11 23]
 [16 72 12]
 [15 13 72]]


In [ ]:
##=== Training Performance ===
print('=== Training Performance ===')
svm_preds_train = svm_classifier.predict(X_train)
print(classification_report(y_train_enc, svm_preds_train))
print(confusion_matrix(y_train_enc, svm_preds_train))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90      5000
           1       0.95      0.88      0.91      5000
           2       0.89      0.93      0.91      5000

    accuracy                           0.91     15000
   macro avg       0.91      0.91      0.91     15000
weighted avg       0.91      0.91      0.91     15000

[[4552  150  298]
 [ 294 4405  301]
 [ 240   96 4664]]


In [ ]:
### IGNORE ####

results = rf.cv_results_
for mean, std, params in zip(results["mean_test_score"],
                              results["std_test_score"],
                              results["params"]):
    print(f"{params} --> mean: {mean:.4f}, std: {std:.4f}")

{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 50} --> mean: 0.5142, std: 0.0085
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100} --> mean: 0.5124, std: 0.0069
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 200} --> mean: 0.5115, std: 0.0076
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50} --> mean: 0.5139, std: 0.0098
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100} --> mean: 0.5113, std: 0.0058
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 200} --> mean: 0.5149, std: 0.0078
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 50} --> mean: 0.5383, std: 0.0079
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100} --> mean: 0.5452, std: 0.0063
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200} --> mean: 0.5

In [ ]:
# --- Color Histogram Only ---
def extract_color_histogram(image, bins=32, resize_dim=(64, 64)):
    image = image.resize(resize_dim).convert('RGB')
    image_np = np.array(image)
    hist_r = np.histogram(image_np[:, :, 0], bins=bins, range=(0, 256))[0]
    hist_g = np.histogram(image_np[:, :, 1], bins=bins, range=(0, 256))[0]
    hist_b = np.histogram(image_np[:, :, 2], bins=bins, range=(0, 256))[0]
    hist = np.concatenate([hist_r, hist_g, hist_b])
    return hist / np.sum(hist)

